In [1]:
import wikipedia
import math
import random
import numpy as np
import pandas as pd


In [2]:

topics = [
    "Artificial Intelligence",
    "Climate Change",
    "Quantum Computing",
    "World War II",
    "Ancient Egypt",
    "Space Exploration",
    "Global Health",
    "Economics",
    "Philosophy of Science",
    "Modern Art",
    "Genetics",
    "Renewable Energy",
    "Cybersecurity",
    "Cryptocurrency",
    "Social Media",
    "Cultural Anthropology",
    "Astrophysics",
    "Human Rights",
    "Machine Learning",
    "History of Technology",
    "Biotechnology"
]

In [39]:
def divide_corpus(corpus):
    sentences=corpus.split('.')
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0]
    return sentences
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokenized = sentence.split(' ')
        final_tokenized=[]
        for word in tokenized:
            if len(word)>0:
                if (word[-1]>='a' and word[-1]<='z') or word[-1]=='>':
                    final_tokenized.append(word)
                else:
                    final_tokenized.append(word[:-1])
        tokenized_sentences.append(final_tokenized)
        
    
    return tokenized_sentences

def get_vocabulary_bigram(data,min_freq=2):
    word_counts={}  #keys:word and value:count
    vocab=[]
    for sentence in data:
        for word in sentence:
            if word in word_counts.keys():
                word_counts[word]+=1
            else:
                word_counts[word]=1
    for sentence in data:
        for word in sentence:
            if word_counts[word]>min_freq and word not in vocab:
                vocab.append(word)
    return vocab,word_counts


def get_vocabulary_trigram(data,min_freq=2):
    word_counts={}  #keys:word and value:count
    vocab=[]
    for sentence in data:
        for i in range(len(sentence)-1):
            if (sentence[i],sentence[i+1]) in word_counts.keys():
                word_counts[(sentence[i],sentence[i+1])]+=1
            else:
                word_counts[(sentence[i],sentence[i+1])]=1
            vocab.append(sentence[i])
            if i==len(sentence)-2:
                vocab.append(sentence[i+1])            
    return vocab,word_counts
def add_starts_ends(data,n=2):
    final_data=[]
    for sentence in data:
        final_data.append(['<s>'] * n + sentence + ['<e>'])
    return final_data
# def replace_with_unk(data,vocab):
#     final_data=[]
#     for sentence in data:
#         final_sentence=[]
#         for word in sentence:
#             if word in vocab:
#                 final_sentence.append(word)
#             else:
#                 final_sentence.append('<unk>')
#         final_data.append(final_sentence)
#     return final_data
def calculate_probability_birgrams(data,vocab_size,word_counts,k=3):
    count_bigrams={}
    for sentence in data:
        for i in range(len(sentence)-1):
            if (sentence[i],sentence[i+1]) in count_bigrams.keys():
                count_bigrams[(sentence[i],sentence[i+1])]+=1
            else:
                count_bigrams[(sentence[i],sentence[i+1])]=1
    probability_bigrams={}
    for key,values in count_bigrams.items():
        probability_bigrams[key]=(count_bigrams[key]+k)/(word_counts[key[0]]+k*vocab_size)
    return probability_bigrams

def calculate_probability_trigrams(data,vocab_size,word_counts,k=3):
    count_trigrams={}
    for sentence in data:
        for i in range(len(sentence)-2):
            if (sentence[i],sentence[i+1],sentence[i+2]) in count_trigrams.keys():
                count_trigrams[(sentence[i],sentence[i+1],sentence[i+2])]+=1
            else:
                count_trigrams[(sentence[i],sentence[i+1],sentence[i+2])]=1
    probability_trigrams={}
    for key,values in count_trigrams.items():
        probability_trigrams[key]=(count_trigrams[key]+k)/(word_counts[(key[0],key[1])]+k*vocab_size)
    return probability_trigrams
def suggest_word(sentence,lm):
    max_prob=-10000
    suggested_word=''
    sentence_log_prob=sum([np.log(lm[(sentence[i],sentence[i+1],sentence[i+2])]) for i in range(len(sentence)-2) if (sentence[i],sentence[i+1],sentence[i+2]) in lm.keys()])
    for key,value in lm.items():
        if key[0]==sentence[len(sentence)-2] and key[1]==sentence[len(sentence)-1]:
            if np.log(value)+sentence_log_prob>max_prob:
                max_prob=np.log(value)+sentence_log_prob
                suggested_word=key[2]
    return suggested_word

In [4]:

corpus=''
for topic in topics:
    try:
        page = wikipedia.page(topic)
        corpus+=page.content
        print(topic)
    except:
        print('skipped:',topic)
        continue

Artificial Intelligence
Climate Change
Quantum Computing
World War II
Ancient Egypt
skipped: Space Exploration
Global Health
Economics
Philosophy of Science
Modern Art


/home/mahdi/anaconda3/envs/myenv/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/mahdi/anaconda3/envs/myenv/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


skipped: Genetics
Renewable Energy
Cybersecurity
Cryptocurrency
Social Media
Cultural Anthropology
Astrophysics
Human Rights
skipped: Machine Learning
History of Technology
Biotechnology


In [16]:
data=divide_corpus(corpus)
data=tokenize_sentences(data)
data=add_starts_ends(data)
vocab,word_counts=get_vocabulary_trigram(data,3)
# data=replace_with_unk(data,vocab)


In [60]:
lm=calculate_probability_trigrams(data,len(vocab),word_counts)
sentence="we are"
sentence=divide_corpus(sentence)
sentence=tokenize_sentences(sentence)
# sentence=add_starts_ends(sentence)
print(suggest_word(sentence[0],lm))

equipped


In [71]:
sentence="<s> philosophy"
prep_sentence=divide_corpus(sentence)
prep_sentence=tokenize_sentences(prep_sentence)
while suggest_word(prep_sentence[0],lm)!="<e>" and suggest_word(prep_sentence[0],lm)!="":
    sentence+=" "+suggest_word(prep_sentence[0],lm)
    prep_sentence=divide_corpus(sentence)
    prep_sentence=tokenize_sentences(prep_sentence)
print(sentence)

<s> philosophy of science
